In [11]:
#Imports the packages we need
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
from getdist import MCSamples

# Constants
SPEED_OF_LIGHT = 2.998e5

# Hubble rate
def Hubble(z, OmegaL, OmegaM, h):
    OmegaK = 1. - OmegaL - OmegaM
    H = (h*100.) * np.sqrt(OmegaL + OmegaM * (1+z)**3 + OmegaK * (1+z)**2)
    return H

# Luminosity distance
def D(z, OmegaL, OmegaM, h):
    return (100 * h) * integrate.quad(lambda x: 1/Hubble(x, OmegaL, OmegaM, h), 0, z)[0]

# Luminosity distance
def dL(z, OmegaL, OmegaM, h):
    OmegaK = 1. - OmegaL - OmegaM
    Dz = D(z, OmegaL, OmegaM, h)

    if OmegaK > 0:
        dL = 1/np.sqrt(OmegaK) * np.sinh(np.sqrt(OmegaK) * Dz)
    elif OmegaK == 0:
        dL = Dz
    else:
        dL = 1/np.sqrt(-OmegaK) * np.sin(np.sqrt(-OmegaK) * Dz)

    dL = 10.**4 * SPEED_OF_LIGHT / h * (1+z) * dL

    return dL

# Distance modulus
def mu(z, OmegaL, OmegaM, h):
    return 5 * np.log10(dL(z, OmegaL, OmegaM, h))

# Read data from URL
dataloc = "http://supernova.lbl.gov/Union/figures/SCPUnion2.1_mu_vs_z.txt"
data = np.genfromtxt(dataloc)
zs = data[:,1] # Redshift
distance_modulus = data[:,2] # Distance modulus
error_distance_modulus = data[:,3] # Error on distance modulus

# Gelman-Rubin convergence test
def gelman_rubin_convergence(chains):
    """
    Computes the Gelman-Rubin convergence diagnostic (potential scale reduction factor).

    Parameters:
        chains (list of arrays): List of MCMC chains, where each chain is represented as an array.

    Returns:
        float: Gelman-Rubin convergence diagnostic value.
    """
    n_chains = len(chains)
    n_samples = chains[0].shape[0]
    n_params = chains[0].shape[1]

    # Compute within-chain variance
    W = np.mean([np.var(chain, axis=0, ddof=1) for chain in chains], axis=0)

    # Compute between-chain variance
    chain_means = [np.mean(chain, axis=0) for chain in chains]
    chain_mean = np.mean(chain_means, axis=0)
    B = n_samples / (n_chains - 1) * np.sum((chain_means - chain_mean)**2, axis=0)

    # Estimate variance of the posterior distribution
    V_hat = (n_samples - 1) / n_samples * W + 1 / n_samples * B

    # Compute potential scale reduction factor (PSRF)
    R_hat = np.sqrt(V_hat / W)

    return R_hat

# Define redshift bins
bins = np.linspace(0, 1.5, 4) # Divide into 3 bins: [0, 0.5], (0.5, 1.0], (1.0, 1.5]
bin_centers = (bins[1:] + bins[:-1]) / 2

# Perform analysis on each bin
for i in range(len(bin_centers)):
    # Select data points within the bin
    mask = (zs > bins[i]) & (zs <= bins[i+1])
    bin_zs = zs[mask]
    bin_distance_modulus = distance_modulus[mask]
    bin_error_distance_modulus = error_distance_modulus[mask]

    # Perform analysis (MCMC, Gelman-Rubin test, etc.) on the bin here...
    # For demonstration, let's print the bin number and some statistics
    print(f"Bin {i+1} Analysis:")
    print(f"Number of data points: {len(bin_zs)}")
    print(f"Average redshift: {np.mean(bin_zs)}")
    print(f"Average distance modulus: {np.mean(bin_distance_modulus)}")
    print(f"Average error on distance modulus: {np.mean(bin_error_distance_modulus)}")
    print()

# MCMC loop
filename = "MC_Chain_SNIa.txt"
file = open(filename, "w")

N_steps = 10000
N_samples = 1000
Omega_M = 0.3
Omega_L = 0.7
h = 0.7

d_Omega_M = 0.01
d_Omega_L = 0.01
d_h = 0.01

chi2_total_best = np.inf

# Pre-compute constant factor outside the loop
h_factor = 100 * h

# Collect samples from MCMC chains
chains = []

for step in range(N_steps):
    Omega_M_new = np.random.normal(Omega_M, d_Omega_M, size=N_samples)
    Omega_L_new = np.random.normal(Omega_L, d_Omega_L, size=N_samples)
    h_new = np.random.normal(h, d_h, size=N_samples)

    chains.append(np.column_stack((Omega_M_new, Omega_L_new, h_new)))

    chi2_total = 0.

    # Initialize array to store mu_model for each z
    mu_models = np.zeros_like(zs)

    for i, z in enumerate(zs):
        # Randomly select a single value from each array
        Omega_M_new_rand = np.random.choice(Omega_M_new)
        Omega_L_new_rand = np.random.choice(Omega_L_new)
        h_new_rand = np.random.choice(h_new)

        # Calculate mu_model for each z
        mu_models[i] = mu(z, Omega_L_new_rand, Omega_M_new_rand, h_new_rand)

    chi2 = np.sum((mu_models - distance_modulus)**2 / error_distance_modulus**2)
    chi2_total += chi2

    likelihood = np.exp(-0.5 * chi2_total)

    if chi2_total < chi2_total_best:
        Omega_M = Omega_M_new
        Omega_L = Omega_L_new
        h = h_new
        chi2_total_best = chi2_total

# Close file
file.close()

# Compute Gelman-Rubin convergence diagnostic
R_hat = gelman_rubin_convergence(chains)
print("Gelman-Rubin convergence diagnostic (R_hat):", R_hat)

# Plotting distance modulus versus redshift for all data points
plt.errorbar(zs, distance_modulus, yerr=error_distance_modulus, fmt='o', markersize=2, color='k', ecolor='g')
plt.xlabel("Redshift")
plt.ylabel("Distance Modulus")
plt.title("Distance Modulus versus Redshift")
plt.show()


Bin 1 Analysis:
Number of data points: 413
Average redshift: 0.18843755767461404
Average distance modulus: 38.65877008396465
Average error on distance modulus: 0.19398526224177554

Bin 2 Analysis:
Number of data points: 138
Average redshift: 0.705936956521739
Average distance modulus: 43.16013663413768
Average error on distance modulus: 0.2970257191040826

Bin 3 Analysis:
Number of data points: 29
Average redshift: 1.1897241379310346
Average distance modulus: 44.63297809548621
Average error on distance modulus: 0.2830429502472759



KeyboardInterrupt: 